loading and processing all .gz files 331

In [1]:
import os
import polars as pl
import glob

# --- Configuration ---
raw_data_folder = "/Users/tusharjoshi/Desktop/ProjectWorkAll/Dissertation /DATA" 

# Define all 15 fields and their target types.
# We no longer need separate top_level_fields_schema and nested_sensor_fields_schema
# as the extraction logic will be unified.
all_fields_target_schema = {
    "check": pl.String,
    "date": pl.String, # Will be parsed to Datetime later
    "id": pl.String,    
    "state": pl.String, 
    "tempBed": pl.Float64,
    "targetBed": pl.Float64,
    "tempNozzle": pl.Float64,
    "targetNozzle": pl.Float64,
    "axisZ": pl.Float64,
    "axisX": pl.Float64,
    "axisY": pl.Float64,
    "flow": pl.Float64,
    "speed": pl.Float64,
    "fanHotend": pl.Float64,
    "fanPrint": pl.Float64,
}

all_fields_order = list(all_fields_target_schema.keys()) # For final selection/ordering


# --- Main Processing Logic ---
all_gz_files = sorted(glob.glob(os.path.join(raw_data_folder, "**/*.gz"), recursive=True))
print(f"Found {len(all_gz_files)} .gz files to process across all subfolders.")

if not all_gz_files:
    print("No .gz files found in the specified folder or its subfolders. Exiting.")
else:
    processed_dfs_list = [] 

    for i, gz_file_path in enumerate(all_gz_files):
        file_name = os.path.basename(gz_file_path)
        print(f"Processing file {i+1}/{len(all_gz_files)}: {file_name}")

        df_raw = pl.read_ndjson(gz_file_path)

        expressions_to_apply = []

        for field_name, field_type in all_fields_target_schema.items():
            if field_name in df_raw.columns:
                # Case 1: Field is top-level
                expressions_to_apply.append(pl.col(field_name).cast(field_type, strict=False))
            elif 'data' in df_raw.columns and isinstance(df_raw.schema['data'], pl.Struct) and field_name in [f.name for f in df_raw.schema['data'].fields]:
                # Case 2: Field is nested in 'data' struct
                expressions_to_apply.append(pl.col("data").struct.field(field_name).alias(field_name).cast(field_type, strict=False))
            else:
                # Case 3: Field is missing from both top-level and 'data' struct
                expressions_to_apply.append(pl.lit(None, dtype=field_type).alias(field_name))
        
        # Apply all expressions to the raw DataFrame
        current_df_processed = df_raw.with_columns(expressions_to_apply)
        
        # Drop the original 'data' column if it exists and is no longer needed
        if 'data' in current_df_processed.columns:
            current_df_processed = current_df_processed.drop("data")

        # Final selection and reordering of all columns
        current_df_processed = current_df_processed.select(all_fields_order)

        processed_dfs_list.append(current_df_processed)
        print(f"  Processed {len(current_df_processed):,} records from {file_name}")

    print(f"\nFinished processing all .gz files. {len(processed_dfs_list)} DataFrames are ready.")

Found 331 .gz files to process across all subfolders.
Processing file 1/331: 2024-04-03-prusa.gz
  Processed 214,770 records from 2024-04-03-prusa.gz
Processing file 2/331: 2024-04-04-prusa.gz
  Processed 477,585 records from 2024-04-04-prusa.gz
Processing file 3/331: 2024-04-05-prusa.gz
  Processed 458,436 records from 2024-04-05-prusa.gz
Processing file 4/331: 2024-04-06-prusa.gz
  Processed 468,234 records from 2024-04-06-prusa.gz
Processing file 5/331: 2024-04-07-prusa.gz
  Processed 472,404 records from 2024-04-07-prusa.gz
Processing file 6/331: 2024-04-09-prusa.gz
  Processed 187,026 records from 2024-04-09-prusa.gz
Processing file 7/331: 2024-04-10-prusa.gz
  Processed 300,461 records from 2024-04-10-prusa.gz
Processing file 8/331: 2024-04-16-prusa.gz
  Processed 151,508 records from 2024-04-16-prusa.gz
Processing file 9/331: 2024-04-17-prusa.gz
  Processed 300,517 records from 2024-04-17-prusa.gz
Processing file 10/331: 2024-04-18-prusa.gz
  Processed 300,475 records from 2024-

Global backfilling and data type enforcement 

Explanation:

fields_to_backfill_globally: This list explicitly names all 15 columns that will have the forward_fill applied.

final_schema_cast: This dictionary defines the final, desired data type for each of your 15 columns after backfilling. Notice date is now pl.Datetime.

pl.concat(processed_dfs_list): This is the critical step that combines all your individual DataFrames into one.

df_combined.sort(["id", "date"]): This is absolutely crucial. It ensures that all records for a given id are grouped together and ordered chronologically. Without this, forward_fill would not work correctly across the entire dataset.

forward_fill().over("id"): Applied to the sorted df_combined, this now allows values to propagate from earlier batches into later batches for the same id.

Final Type Enforcement: We re-apply cast for all columns. For date, we use strptime to convert the string dates (from initial loading) into proper Polars Datetime objects. strict=False is used for all casts to ensure that any values that still cannot be converted (e.g., truly invalid numbers after backfilling) become NULLs instead of crashing the process.

In [2]:

# --- Define fields to backfill globally ---
# These are the fields where you want forward_fill to apply globally per ID
fields_to_backfill_globally = [
    "check", "state", "tempBed", "targetBed", "tempNozzle", "targetNozzle",
    "axisZ", "axisX", "axisY", "flow", "speed", "fanHotend", "fanPrint"
]

# --- Define all 15 fields and their target types for final casting ---
# This ensures final consistency after backfilling
final_schema_cast = {
    "check": pl.String,
    "date": pl.Datetime, # Now we will cast to actual Datetime object
    "id": pl.String,    
    "state": pl.String, 
    "tempBed": pl.Float64,
    "targetBed": pl.Float64,
    "tempNozzle": pl.Float64,
    "targetNozzle": pl.Float64,
    "axisZ": pl.Float64,
    "axisX": pl.Float64,
    "axisY": pl.Float64,
    "flow": pl.Float64,
    "speed": pl.Float64,
    "fanHotend": pl.Float64,
    "fanPrint": pl.Float64,
}

# --- Main Processing Logic for Global Backfilling ---
if not processed_dfs_list:
    print("No DataFrames found in 'processed_dfs_list'. Please run Step 1 first.")
else:
    print("Concatenating all pre-processed DataFrames...")
    df_combined = pl.concat(processed_dfs_list)
    print(f"Combined DataFrame loaded successfully with {df_combined.shape[0]:,} rows.")

    # --- Apply global backfilling across the entire combined DataFrame ---
    print("Sorting DataFrame by ID and Date for global forward-fill...")
    # It's crucial to sort by ID and then Date for correct forward-fill within each ID group
    df_combined = df_combined.sort(["id", "date"]) 
    print("Sorting complete.")

    print("Applying global forward-fill across all batches for each ID...")
    df_combined = df_combined.with_columns([
        pl.col(field).forward_fill().over("id")
        for field in fields_to_backfill_globally
    ])
    print("Global forward-fill complete.")

    # --- Final Data Type Enforcement ---
    # Re-cast 'date' to actual Datetime object and ensure all other types are final
    print("Enforcing final data types on the combined DataFrame...")
    for col_name, col_type in final_schema_cast.items():
        # For 'date' column, we use strptime to parse from string to Datetime
        if col_name == "date" and col_type == pl.Datetime:
            df_combined = df_combined.with_columns(
                pl.col(col_name).str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.f%Z", strict=False).alias(col_name)
            )
        else:
            df_combined = df_combined.with_columns(
                pl.col(col_name).cast(col_type, strict=False) # strict=False converts unparseable to NULL
            )
    print("Final data type enforcement complete.")

    # The 'df_combined' DataFrame now holds your fully backfilled and type-enforced data.
    # We will use this DataFrame in the next step.
    print(f"\nStep 2 complete. Final DataFrame has {df_combined.shape[0]:,} rows and {len(df_combined.columns)} columns.")

Concatenating all pre-processed DataFrames...
Combined DataFrame loaded successfully with 94,869,699 rows.
Sorting DataFrame by ID and Date for global forward-fill...
Sorting complete.
Applying global forward-fill across all batches for each ID...
Global forward-fill complete.
Enforcing final data types on the combined DataFrame...
Final data type enforcement complete.

Step 2 complete. Final DataFrame has 94,869,699 rows and 15 columns.


let us check the null values for each column now 

In [3]:

print("\n--- Verifying Nulls in the Globally Backfilled DataFrame ---")
print(f"{'Field':<20} {'Total Nulls':<15} {'Percentage':<10}")
print("-" * 50)

total_records_in_df_combined = df_combined.shape[0]

# Iterate through all columns in the combined DataFrame
for col_name in df_combined.columns:
    null_count = df_combined[col_name].null_count()
    
    null_pct = (null_count / total_records_in_df_combined * 100) if total_records_in_df_combined > 0 else 0
    
    print(f"{col_name:<20}: {null_count:<15,} ({null_pct:<6.2f}%)")

print("-" * 50)
print(f"Total records processed: {total_records_in_df_combined:,}")

# Store the results for later reference if needed
global_null_summary = {
    col_name: {"null_count": df_combined[col_name].null_count(), 
               "null_percentage": (df_combined[col_name].null_count() / total_records_in_df_combined * 100) if total_records_in_df_combined > 0 else 0}
    for col_name in df_combined.columns
}


--- Verifying Nulls in the Globally Backfilled DataFrame ---
Field                Total Nulls     Percentage
--------------------------------------------------
check               : 0               (0.00  %)
date                : 0               (0.00  %)
id                  : 0               (0.00  %)
state               : 0               (0.00  %)
tempBed             : 0               (0.00  %)
targetBed           : 0               (0.00  %)
tempNozzle          : 0               (0.00  %)
targetNozzle        : 0               (0.00  %)
axisZ               : 0               (0.00  %)
axisX               : 0               (0.00  %)
axisY               : 0               (0.00  %)
flow                : 0               (0.00  %)
speed               : 0               (0.00  %)
fanHotend           : 0               (0.00  %)
fanPrint            : 0               (0.00  %)
--------------------------------------------------
Total records processed: 94,869,699


perfect now we export it to mysql 


In [4]:
import os
import polars as pl
import pymysql
from sqlalchemy import create_engine, text



# --- Configuration ---
# MySQL Database Connection Details

DB_HOST = "localhost"
DB_USER = "root"       # Use 'root' if you skipped creating a dedicated user
DB_PASSWORD = "admintushar15" # Replace with your MySQL root password
DB_NAME = "printer_data_db" # The database name you created in DBeaver
TABLE_NAME_IN_DB = "PrinterData" # Name of the table to create in MySQL

batch_size = 100000 # Define a reasonable batch size for insertion (e.g., 100,000 rows)

# --- Main Export Process to MySQL ---
try:
    # Connect to MySQL using pymysql
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
    cursor = conn.cursor()

    print(f"\nExporting data to MySQL database '{DB_NAME}' table '{TABLE_NAME_IN_DB}' in batches...")

    # Drop table if it exists to ensure a clean export
    cursor.execute(f"DROP TABLE IF EXISTS `{TABLE_NAME_IN_DB}`;")
    
    # Create table schema in MySQL based on Polars DataFrame columns and types
    type_mapping = {
        pl.String: "TEXT",
        pl.Float64: "DOUBLE", # MySQL uses DOUBLE for floating point numbers
        pl.Datetime: "DATETIME" # MySQL DATETIME type
    }
    columns_sql_defs = []
    for col_name, dtype in df_combined.schema.items():
        mysql_type = type_mapping.get(dtype, "TEXT") 
        columns_sql_defs.append(f'`{col_name}` {mysql_type}') 
    
    create_table_sql = f"CREATE TABLE `{TABLE_NAME_IN_DB}` ({', '.join(columns_sql_defs)});"
    cursor.execute(create_table_sql)
    print(f"Created table '{TABLE_NAME_IN_DB}' in MySQL.")

    # Prepare the INSERT statement
    placeholders = ", ".join(["%s" for _ in df_combined.columns]) # Use %s for pymysql placeholders
    insert_sql = f"INSERT INTO `{TABLE_NAME_IN_DB}` VALUES ({placeholders});"

    # Iterate over the Polars DataFrame in batches and insert
    total_inserted_rows = 0
    for batch_start in range(0, df_combined.shape[0], batch_size):
        batch_end = min(batch_start + batch_size, df_combined.shape[0])
        df_batch_to_insert = df_combined.slice(batch_start, batch_end - batch_start)
        
        # Convert Polars batch to a list of tuples for pymysql.executemany
        rows_to_insert = []
        for row_data in df_batch_to_insert.iter_rows():
            formatted_row = []
            for col_val, col_name in zip(row_data, df_combined.columns): # Iterate with col_name to check type
                if isinstance(col_val, pl.Datetime):
                    formatted_row.append(col_val.isoformat()) # Convert datetime to ISO string
                else:
                    formatted_row.append(col_val)
            rows_to_insert.append(tuple(formatted_row))

        cursor.executemany(insert_sql, rows_to_insert)
        conn.commit() # Commit after each batch
        total_inserted_rows += len(rows_to_insert)
        print(f"  Inserted {len(rows_to_insert)} rows. Total inserted: {total_inserted_rows}")

    print(f"Successfully exported all data to MySQL table '{TABLE_NAME_IN_DB}'.")

except Exception as e:
    print(f"\nAn error occurred during the MySQL export process: {e}")
    print("Please check:")
    print("- MySQL server is running and accessible.")
    print("- Database name, username, and password are correct.")
    print("- You have sufficient privileges to create/write to the table.")
    print("- That you have enough disk space on your MySQL server.")

finally:
    if 'conn' in locals() and conn.open: 
        conn.close()
        print("MySQL connection closed.")


Exporting data to MySQL database 'printer_data_db' table 'PrinterData' in batches...
Created table 'PrinterData' in MySQL.
  Inserted 100000 rows. Total inserted: 100000
  Inserted 100000 rows. Total inserted: 200000
  Inserted 100000 rows. Total inserted: 300000
  Inserted 100000 rows. Total inserted: 400000
  Inserted 100000 rows. Total inserted: 500000
  Inserted 100000 rows. Total inserted: 600000
  Inserted 100000 rows. Total inserted: 700000
  Inserted 100000 rows. Total inserted: 800000
  Inserted 100000 rows. Total inserted: 900000
  Inserted 100000 rows. Total inserted: 1000000
  Inserted 100000 rows. Total inserted: 1100000
  Inserted 100000 rows. Total inserted: 1200000
  Inserted 100000 rows. Total inserted: 1300000
  Inserted 100000 rows. Total inserted: 1400000
  Inserted 100000 rows. Total inserted: 1500000
  Inserted 100000 rows. Total inserted: 1600000
  Inserted 100000 rows. Total inserted: 1700000
  Inserted 100000 rows. Total inserted: 1800000
  Inserted 100000 row